In [17]:
from keras_tuner.tuners import RandomSearch
import pandas as pd
import numpy as np



In [18]:
data=pd.read_csv('Real_Combine.csv')
data

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4,219.720833
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1,182.187500
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1,154.037500
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6,223.208333
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2,200.645833
...,...,...,...,...,...,...,...,...,...
1088,18.1,24.0,11.2,1015.4,56.0,1.8,15.9,25.9,288.416667
1089,17.8,25.0,10.7,1015.8,54.0,2.3,9.4,22.2,256.833333
1090,13.9,24.5,11.4,1015.0,95.0,0.6,8.7,14.8,169.000000
1091,16.3,23.0,9.8,1016.9,78.0,1.1,7.4,16.5,186.041667


In [19]:
X=data.iloc[:,:-1]
y=data.iloc[:,-1]

In [20]:
X.isna().sum()
y.isna().sum()



np.int64(1)

In [21]:
from sklearn.preprocessing import StandardScaler


scaler = StandardScaler()
X = scaler.fit_transform(X)

In [22]:
y.fillna(y.mean(),inplace=True)

In [23]:
from tensorflow.keras import layers
from tensorflow import keras

In [36]:
def build_model(hp):
    model=keras.Sequential()
    model.add(keras.Input(shape=(X_train.shape[1],)))

    for i in range(1, hp.Int('num_layers', min_value=2, max_value=8, step=1)):
        model.add(layers.Dense(units=hp.Int('units_'+str(i),min_value=32,max_value=512,step=32),activation='relu'))
    model.add(layers.Dense(1,activation='linear'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate',[1e-2,1e-3,1e-4]),
            
        ),
        loss='mean_absolute_error',
        metrics=['mean_absolute_error']

    )
    return model


In [25]:
'''def build_model(hp):
    model = keras.Sequential()

    # First layer needs input_shape
    model.add(
        layers.Dense(
            units=hp.Int('units_0', min_value=32, max_value=512, step=32),
            activation='relu',
            input_shape=(X_train.shape[1],)   # 👈 important
        )
    )

    # Additional hidden layers
    for i in range(1, hp.Int('num_layers', 2, 5)):  # keep search smaller at first
        model.add(
            layers.Dense(
                units=hp.Int('units_' + str(i), min_value=32, max_value=512, step=32),
                activation='relu'
            )
        )

    # Output layer for regression
    model.add(layers.Dense(1, activation='linear'))

    # Compile
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])
        ),
        loss='mean_absolute_error',
        metrics=['mean_absolute_error']
    )

    return model
'''

"def build_model(hp):\n    model = keras.Sequential()\n\n    # First layer needs input_shape\n    model.add(\n        layers.Dense(\n            units=hp.Int('units_0', min_value=32, max_value=512, step=32),\n            activation='relu',\n            input_shape=(X_train.shape[1],)   # 👈 important\n        )\n    )\n\n    # Additional hidden layers\n    for i in range(1, hp.Int('num_layers', 2, 5)):  # keep search smaller at first\n        model.add(\n            layers.Dense(\n                units=hp.Int('units_' + str(i), min_value=32, max_value=512, step=32),\n                activation='relu'\n            )\n        )\n\n    # Output layer for regression\n    model.add(layers.Dense(1, activation='linear'))\n\n    # Compile\n    model.compile(\n        optimizer=keras.optimizers.Adam(\n            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])\n        ),\n        loss='mean_absolute_error',\n        metrics=['mean_absolute_error']\n    )\n\n    return model\n"

In [37]:
tuner= RandomSearch(
    build_model,
    objective='val_mean_absolute_error',
    max_trials=5,
    executions_per_trial=3,
    directory='project',
    project_name='air quality index'
)

In [27]:
tuner.search_space_summary()

Search space summary
Default search space size: 20
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': 'linear'}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}
units_2 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
units_3 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
units_4 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
units_5 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
units_6

In [28]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25)

In [29]:
import tensorflow as tf
print(tf.constant(1) + tf.constant(2))


tf.Tensor(3, shape=(), dtype=int32)


In [30]:
import tensorflow as tf
import keras_tuner as kt

print(tf.__version__)
print(kt.__version__)


2.20.0-rc0
1.4.7


In [38]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 5 Complete [00h 00m 07s]
val_mean_absolute_error: 38.7613271077474

Best val_mean_absolute_error So Far: 37.5421511332194
Total elapsed time: 00h 00m 38s


In [45]:
# Get the single best hyperparameter configuration
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Print the optimal values
print(f"""
The hyperparameter search is complete. The optimal number of layers is {best_hps.get('num_layers')}
and the optimal learning rate for the optimizer is {best_hps.get('learning_rate')}.
""")

# You can also print the optimal number of units for each layer
for i in range(best_hps.get('num_layers')):
    print(f"Optimal units for layer {i}: {best_hps.get('units_' + str(i))}")


The hyperparameter search is complete. The optimal number of layers is 3
and the optimal learning rate for the optimizer is 0.01.



KeyError: 'units_0 does not exist.'

In [42]:
tuner.get_best_models()

f:\aiml2\.venv\Lib\site-packages\keras\src\saving\saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


[<Sequential name=sequential, built=True>]

In [46]:
tuner.results_summary()

Results summary
Results in project\air quality index
Showing 10 best trials
Objective(name="val_mean_absolute_error", direction="min")

Trial 2 summary
Hyperparameters:
num_layers: 3
units_1: 128
learning_rate: 0.01
units_2: 160
Score: 37.5421511332194

Trial 4 summary
Hyperparameters:
num_layers: 2
units_1: 288
learning_rate: 0.01
units_2: 128
Score: 38.7613271077474

Trial 3 summary
Hyperparameters:
num_layers: 2
units_1: 192
learning_rate: 0.01
units_2: 288
Score: 39.128379821777344

Trial 1 summary
Hyperparameters:
num_layers: 2
units_1: 384
learning_rate: 0.001
units_2: 64
Score: 91.14804077148438

Trial 0 summary
Hyperparameters:
num_layers: 3
units_1: 32
learning_rate: 0.001
units_2: 32
Score: 93.55663045247395


In [47]:
# Get the best model from the search
best_model = tuner.get_best_models(num_models=1)[0]

# You can now use this model as a standard Keras model
best_model.summary()

f:\aiml2\.venv\Lib\site-packages\keras\src\saving\saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │         1,152 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 160)            │        20,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │           161 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,953 (85.75 KB)

 Trainable params: 21,953 (85.75 KB)

 Non-trainable params: 0 (0.00 B)